### Automation Script Overhaul 
For automating the parsing, transformation, and geographic file creation from .GPX<br><br>
Work taken from the script by Isias (ler_gpx.py) and workflow by Simone. Further developments by Kyle & Isais. <br><br>
Data collection must be done with Locus Map 4.x or formatted similarly to GPX files exportd by Locus Map 4 for this script to work.

### Section 1
This section takes the raw GPX files and makes them ready for QGIS and statistical analysis.

In [ ]:
import gpxpy
import pandas as pd
import geopandas as gpd
import os
from os import mkdir
from datetime import datetime, timedelta

# Create a dictionary with all gpx files in dictionary / Crie um dicionário com todos os arquivos gpx no dicionário

gpxDict = dict()

for file in os.listdir():
    if file.endswith('.gpx'):
       gpxDict[file] = 'file_'+file
gpxDict

##### Daily Data
This next for loop / cell will create files with the data sorted by day and must be run before creaating data files by scan.

In [1]:
import gpxpy
import pandas as pd
import geopandas as gpd
import warnings
import os
from os import mkdir
from datetime import datetime, timedelta

import spatialFunctions
from spatialFunctions import *

# Remove warning message for future warnings / Remover mensagem de aviso para avisos futuros
warnings.filterwarnings(action='ignore',category=FutureWarning)
# Suppress warning for setting with copy, non-issue here / Suprimir aviso para configuração com cópia, não é problema aqui
pd.set_option('mode.chained_assignment',None)

# Get user decision for input fields and state the variables / Obtenha a decisão do usuário para campos de entrada e indique as variáveis
userInput = input('To add observer, group, and weather information for each day type \'yes\'. Otherwise hit escape')
observer = ''
group = ''
weather = ''

# Create a dictionary with all gpx files in dictionary / Crie um dicionário com todos os arquivos gpx no dicionário

gpxDict = dict()

for file in os.listdir():
    if file.endswith('.gpx'):
       gpxDict[file] = 'file_'+file

# Run for loop to cover every gpx file in directory / Execute o loop para cobrir todos os arquivos gpx no diretório

for i in gpxDict:
    
    # Open and read in the .gpx to a dataframe / Abra e leia no .gpx para um dataframe
    gpxCurrent = i
    gpxCurrent = open(gpxCurrent)
    gpxCurrent = gpxpy.parse(gpxCurrent)
    gpxCurrent = gpxCurrent.to_xml()
    df = pd.read_xml(gpxCurrent)

    # Ask for observer, group, climate conditions / Pergunte por observador, grupo, condições climáticas
    if userInput:
        observer = input('Input for '+i+': Observer/Observador? ')
        group = input('Input for '+i+': Group/Grupo? (if both, mark 0) ') # or leave blank?
        weather = input('Input for '+i+': Weather conditions/Condição do clima? ')
    
    # Remove unecessary columns / Remova colunas desnecessárias
    df.pop('desc')
    df.pop('time')
    if 'hdop' in df.columns:
        df.pop('hdop')
    df = df.drop(index=0)

    # Reorganize columns / Reorganizar colunas
    shiftPos = df.pop('name')
    df.insert(0, 'name', shiftPos)

    # Insert user input columns if they have a value / Insira colunas de entrada do usuário se elas tiverem um valor
    if observer:
        df.insert(loc=1, column='observer', value=observer, allow_duplicates=True)
    if group:
        df.insert(loc=1, column='group', value=group, allow_duplicates=True)
    if weather:
        df.insert(loc=1, column='weather', value=weather, allow_duplicates=True)

    # Split 'name' into date, time, and observations / Dividir 'nome' em data, hora e observações
    date = df['name'].str[:10]
    df.insert(loc=0, column='date', value=date, allow_duplicates=True)

    time = df['name'].str[11:19]
    df.insert(loc=1, column='time', value=time, allow_duplicates=True)

    obs = df['name'].str[19:]
    df.insert(loc=2, column='obs', value=obs, allow_duplicates=True)
    # Remove whitespace from observations column / Remover espaço em branco da coluna de observações
    df['obs'] = df['obs'].str.strip()

    df.pop('name')

    timeScan(df)
    



    # # Create list for temporary storage of scan ID's / Criar lista para armazenamento temporário de IDs de digitalização
    # scanNum = []

    # # Loop to check each time against the times for each day and assign scan ID
    # # Faça um loop para verificar cada vez em relação aos horários de cada dia e atribuir a ID de verificação
    # for row in df['time']:
    #     row = datetime.strptime(row,'%H:%M:%S')
    #     if scanStart <= row <= scanEnd:
    #         scanNum.append('1')
    #     elif (scanStart + timedelta(minutes=scanMins*2-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*2+bufferMins)):
    #         scanNum.append('2')
    #     elif (scanStart + timedelta(minutes=scanMins*4-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*4+bufferMins)):
    #         scanNum.append('3')
    #     elif (scanStart + timedelta(minutes=scanMins*6-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*6+bufferMins)):
    #         scanNum.append('4')
    #     elif (scanStart + timedelta(minutes=scanMins*8-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*8+bufferMins)):
    #         scanNum.append('5')
    #     elif (scanStart + timedelta(minutes=scanMins*10-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*10+bufferMins)):
    #         scanNum.append('6')
    #     elif (scanStart + timedelta(minutes=scanMins*12-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*12+bufferMins)):
    #         scanNum.append('7')
    #     elif (scanStart + timedelta(minutes=scanMins*14-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*14+bufferMins)):
    #         scanNum.append('8')
    #     elif (scanStart + timedelta(minutes=scanMins*16-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*16+bufferMins)):
    #         scanNum.append('9')
    #     elif (scanStart + timedelta(minutes=scanMins*18-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*18+bufferMins)):
    #         scanNum.append('10')
    #     elif (scanStart + timedelta(minutes=scanMins*20-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*20+bufferMins)):
    #         scanNum.append('11')
    #     elif (scanStart + timedelta(minutes=scanMins*22-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*22+bufferMins)):
    #         scanNum.append('12')
    #     elif (scanStart + timedelta(minutes=scanMins*24-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*24+bufferMins)):
    #         scanNum.append('13')
    #     elif (scanStart + timedelta(minutes=scanMins*26-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*26+bufferMins)):
    #         scanNum.append('14')
    #     elif (scanStart + timedelta(minutes=scanMins*28-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*28+bufferMins)):
    #         scanNum.append('15')
    #     elif (scanStart + timedelta(minutes=scanMins*30-bufferMins)) <= row <= (scanEnd + timedelta(minutes=scanMins*30+bufferMins)):
    #         scanNum.append('16')

    #     # If no times fit, apply N/A / Se nenhum tempo se encaixar, aplique N/A
    #     else:
    #         scanNum.append('') 

    # # Apply scan ID list to the dataframe / Aplicar lista de IDs de varredura ao dataframe
    # df['scan'] = scanNum

    # Run the observations method in spatialFunctions / Execute o método de observações em spaceFunctions
    observations(df)

    # Make geographic and set CRS / Faça geográfica e defina CRS
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat)) 
    gdf = gdf.set_crs('EPSG:4326')
    gdf = gdf.to_crs('EPSG:31985')

    # Check and create save directory for gpkg files / Verifique e crie um diretório de salvamento para arquivos gpkg
    gpkgsavePath = './gpkgData'
    isDir = os.path.isdir(gpkgsavePath)
    if isDir == False:
        mkdir('gpkgData')
    
    # Export gdf into gpkg / Exportar gdf para gpkg
    gdf.to_file('gpkgData/'+i[:-4]+'scans.gpkg', driver="GPKG", layer=i[:-4]+'_wholeDay')
    
    # Export each scan as a separate layer using the scanSpatial method in spatialFunctions
    # Exporte cada varredura como uma camada separada usando o método scanSpatial em spaceFunctions
    spatialCounter = '1'
    gdfs1 = gdf[(gdf['scan'].isin(['1']))]
    if not gdfs1.empty:
        scanSpatial(gdfs1, i, '1')

    gdfs2 = gdf[(gdf['scan'].isin(['2']))]
    if not gdfs2.empty:
        scanSpatial(gdfs2, i, '2')

    gdfs3 = gdf[(gdf['scan'].isin(['3']))]
    if not gdfs3.empty:
        scanSpatial(gdfs3, i, '3')

    gdfs4 = gdf[(gdf['scan'].isin(['4']))]
    if not gdfs4.empty:
        scanSpatial(gdfs4, i, '4')

    gdfs5 = gdf[(gdf['scan'].isin(['5']))]
    if not gdfs5.empty:
        scanSpatial(gdfs5, i, '5')

    gdfs6 = gdf[(gdf['scan'].isin(['6']))]
    if not gdfs6.empty:
        scanSpatial(gdfs6, i, '6')

    gdfs7 = gdf[(gdf['scan'].isin(['7']))]
    if not gdfs7.empty:
            scanSpatial(gdfs7, i, '7')

    gdfs8 = gdf[(gdf['scan'].isin(['8']))]
    if not gdfs8.empty:
        scanSpatial(gdfs8, i, '8')

    gdfs9 = gdf[(gdf['scan'].isin(['9']))]
    if not gdfs9.empty:
        scanSpatial(gdfs9, i, '9')

    gdfs10 = gdf[(gdf['scan'].isin(['10']))]
    if not gdfs10.empty:
        scanSpatial(gdfs10, i, '10')

    # Create layers for non-scan data / Crie camadas para dados não digitalizados
    gdfago = gdf[(gdf['scan'].isin(['ago']))]
    if not gdfago.empty:
        gdfago.to_file('gpkgData/'+i[:-4]+'scans.gpkg', driver="GPKG", layer=i[:-4]+'_ago')

    gdfother = gdf[(gdf['scan'].isin(['other']))]
    if not gdfother.empty:
        gdfother.to_file('gpkgData/'+i[:-4]+'scans.gpkg', driver="GPKG", layer=i[:-4]+'_other')

    # Check and create save directory for csv files / Verifique e crie um diretório de salvamento para arquivos csv
    csvsavePath = './csvDayFiles'
    isDir = os.path.isdir(csvsavePath)
    if isDir == False:
        mkdir('csvDayFiles')
    
    # Save to csv / Salvar em csv
    gdf.to_csv('csvDayFiles/'+i[:-4]+'.csv')

In [ ]:

        # Calculate the centroid of the group
        centroid = gdfs1.dissolve().centroid
        
        # Calculate distance of each point to the centroid of the group
        for row in gdfs1['geometry']:
            gdfs1.loc[:,'distCentr'] = gdfs1.distance(centroid[0])

        # Create geodataframe for the area, perimeter, and polygon of each scan
        area = gdfs1.dissolve().convex_hull
        area = gpd.GeoDataFrame(gpd.GeoSeries(area))
        area = area.rename(columns={0:'geometry'}).set_geometry('geometry')
        area.loc[:,'area'] = area.area
        area.loc[:,'perimeter'] = area.length
        centroid.to_file('gpkgData/'+i[:-4]+'scans.gpkg', driver="GPKG", layer=i[:-4]+'_scan1_centroid')
        area.to_file('gpkgData/'+i[:-4]+'scans.gpkg', driver="GPKG", layer=i[:-4]+'_scan1_area')

Testing using gdf's for each scan in above loop

In [ ]:
# Calculate the centroid of the group
centroid = gdfs1.dissolve().centroid
centroid

In [ ]:
# Suppress warning for setting with copy, non-issue here
pd.set_option('mode.chained_assignment',None)

# Calculate distance of each point to the centroid of the group
for row in gdfs1['geometry']:
    gdfs1.loc[:,'distCentr'] = gdfs1.distance(centroid[0])
gdfs1.head()

In [ ]:
# Create geodataframe for the area, perimeter, and polygon of each scan
area = gdfs1.dissolve().convex_hull
area = gpd.GeoDataFrame(gpd.GeoSeries(area))
area = area.rename(columns={0:'geometry'}).set_geometry('geometry')
area.loc[:,'area'] = area.area
area.loc[:,'perimeter'] = area.length
area

In [ ]:
area.to_file('gpkgData/20220612scans.gpkg', driver="GPKG", layer='area')

In [ ]:
centroid.to_file('gpkgData/20220612scans.gpkg', driver="GPKG", layer='centroid')

#### Section 2
Analysis of scans, analyze all of this for every individual scan

#### For GPKG data stored in gpkgData folder

In [ ]:
import pandas as pd
import geopandas as gpd

gpkgDict = dict()

for file in os.listdir('gpkgData'):
    if file.endswith('.gpkg'):
       gpkgDict[file] = 'file_'+file




# for i in gpkgDict:


In [ ]:
# Distance between each centroid in temporal order


In [ ]:
# Subgroups/cluster analysis, find clusters on eah scan and distance from each sub-centroid to main group centroid


#### Export Data

In [ ]:
# Export data frame to gpkg for use in QGIS / Exportar quadro de dados para gpkg para uso no QGIS
